### Planteamiento del problema

La enfermedad de Alzheimer es un trastorno neurodegenerativo y la forma más común de demencia. La predicción precisa y el diagnóstico de la enfermedad de Alzheimer (EA) y su etapa prodromal, como el deterioro cognitivo leve, son esenciales para la detección temprana. Uno de los métodos clave para diagnosticar esta enfermedad y monitorear su progresión es a través de la resonancia magnética (RM), que proporciona información detallada sobre la estructura y función del cerebro.

El objetivo principal de esta investigación es desarrollar y evaluar un modelo de **Deep Learning** capaz de identificar patrones distintivos en imágenes de RM que permitan discriminar entre diferentes etapas de pacientes con Alzheimer. Esto podría tener un impacto significativo en la detección temprana de la enfermedad, facilitando así intervenciones más efectivas y mejorando la calidad de vida de los pacientes.

### Contenido

Los datos consisten en imágenes de resonancia magnética (RM). Los datos tienen cuatro clases de imágenes tanto en el conjunto de entrenamiento como en el de pruebas:

- Demencia leve
- Demencia moderada
- No demente
- Demencia muy leve

Los conjuntos de datos se extraen del Proyecto de Series de Estudios de Imágenes de Acceso Abierto (OASIS, por sus siglas en inglés) (https://www.oasis-brains.org), que es un proyecto destinado a poner conjuntos de datos de neuroimagen del cerebro libremente disponibles para la comunidad científica.

#### Metodología

Se han seguido los siguientes pasos para preparar las imágenes para el análisis. Se ha desarrollado un modelo convolucional desde cero que se comparó con algunos modelos en los que se ha utilizado Transfer Learning. Para ello, se ha desarrollado un conjunto de funciones auxiliares para agilizar tareas para su uso repetido durante el análisis.

+ **1. Preparación de datos**
    + Importar bibliotecas
    + Construcción de funciones auxiliares
    + Preprocesamiento de imágenes para el análisis
    + Visualización de imágenes de muestra
    + Visualización de la proporción de clases
+ **2. Comparación de modelos**
    + Modelo base: Random Forest
    + Modelo convolucional simple (modelo CNN)
    + Transfer Learning:
        - ResNet50
        - InceptionV3
        - DenseNet169
        - VGG16
    + Resumen y conclusiones, seleccionar el mejor modelo para ajustar
+ **3. Ajuste del modelo**
    + Fine tuning
    + Aumento de datos (Data augmentation)
+ **4. Evaluación del rendimiento en imágenes de prueba**
+ **5. Conclusiones**

## 1. Preparación de datos

#### Importar bibliotecas

En primer lugar, vamos a importar todas las bibliotecas necesarias para realizar el proyecto.

In [ ]:
# Import Libraries
import pandas as pd 
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import warnings
import scipy
import random

# System libraries
import os
import os.path
from   os import path
import shutil
from pathlib import Path
from distutils.dir_util import copy_tree, remove_tree
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
import time

from PIL import Image
from random import randint

# Metrics
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from collections import Counter
from sklearn.model_selection import cross_val_score
from tensorflow.keras.metrics import AUC, Precision, Recall

# Images, Plotting
from skimage import io
import tensorflow.keras.preprocessing

# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from skimage.io import imread
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from time import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import backend, models, layers, Sequential
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Add
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.applications import DenseNet121,InceptionV3, Xception, ResNet101, VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import load_model
import torch
from torchvision import datasets

# Turn off warnings for cleaner looking notebook
warnings.simplefilter('ignore')


#### Contrucción de funciones auxiliares

Esta funciones nos ayudarán a agilizar tareas para su uso repetido durante el análisis.

In [ ]:
# Esta función es esencialmente un cargador de datos para tareas de clasificación de imágenes.
# Lee imágenes de un directorio, las redimensiona a una dimensión especificada y las recopila junto con sus etiquetas correspondientes.

def read_data(directory, reshape_dim=(176, 176)):
    X = []
    y = []
    for folder in os.listdir(directory):
        if os.path.isdir(os.path.join(directory, folder)):
            label = folder
            for file in os.listdir(os.path.join(directory, folder)):
                image_path = os.path.join(directory, folder, file)
                if os.path.isfile(image_path):
                    image = cv2.imread(image_path)
                    image = cv2.resize(image, reshape_dim)
                    X.append(image)
                    y.append(label)
    return np.array(X), np.array(y)

In [ ]:
# Esta función es para visualizar un conjunto de imágenes aleatorias con sus etiquetas 
# correspondientes para realizar una inspección rápida del conjunto de datos.

def show_random_images_with_labels(X, y, num_images=10, rows=2, cols=5):
    fig, axes = plt.subplots(rows, cols, figsize=(10, 6))
    indices = np.random.choice(len(X), num_images, replace=False)
    for idx, ax in zip(indices, axes.ravel()):
        image = X[idx]
        label = y[idx]
        ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        ax.set_title(f"Label: {label}")
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Esta función es para preparar datos de imagen para modelos de aprendizaje automático que 
# requieren vectores de características aplanados como entrada. 
# Además, proporciona la opción de convertir imágenes a escala de grises, lo cual puede ser 
# beneficioso para reducir la complejidad computacional.

def flatten_gray_scale(array_in, gray=True):
    def convert_to_gray_scale(color_image):
        coefficients = np.array([0.2989, 0.5870, 0.1140])
        gray_image = np.dot(color_image[..., :3], coefficients)
        return gray_image

    if array_in.shape[-1] == 3 and gray:
        array_in = np.array([convert_to_gray_scale(image) for image in array_in])
    return array_in.reshape(array_in.shape[0], -1)

In [ ]:
# Esta función analiza el rendimiento de un modelo durante las fases de entrenamiento y validación, 
# así como proporciona información sobre su rendimiento en datos de prueba no vistos a través de métricas y visualizaciones. 
# Además, proporciona una matriz de confusión para comprender las predicciones por clase.

def plot_training_metrics(train_hist, model, test_gen_plot, y_actual, y_pred, classes, model_name):
    """
    Entrada: historial del modelo entrenado, modelo, generador de imágenes de prueba, etiquetas reales y predichas, lista de clases
    Salida: Gráficos de pérdida vs épocas, precisión vs épocas, matriz de confusión
    """
    
    # Evaluar los resultados:
    test_metrics = model.evaluate(test_gen_plot, verbose=False)
    AUC = test_metrics[1] * 100
    Acc = test_metrics[2] * 100 
    resultados_título = f"\n Modelo AUC {AUC:.2f}%, Precisión {Acc:.2f}% en Datos de Prueba\n"
    print(resultados_título.format(AUC, Acc))

    # imprimir informe de clasificación
    print(classification_report(y_actual, y_pred, target_names=classes))

    # extraer datos del historial de entrenamiento para trazar
    history_dict = train_hist.history
    valores_pérdida = history_dict['loss']
    val_valores_pérdida = history_dict['val_loss']
    valores_auc = history_dict['auc']
    val_valores_auc = history_dict['val_auc']
    épocas = range(1, len(history_dict['auc']) + 1)

    # obtener la pérdida mínima y la máxima precisión para trazar
    máx_auc = np.max(val_valores_auc)
    mín_pérdida = np.min(val_valores_pérdida)
    
    # Establecer el fondo a gris
    plt.style.use('ggplot')
    
    # crear gráficos
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    
    # trazar pérdida por épocas
    axs[0].plot(épocas, valores_pérdida, 'bo', label='Pérdida de entrenamiento')
    axs[0].plot(épocas, val_valores_pérdida, 'cornflowerblue', label='Pérdida de validación')
    axs[0].set_title('Pérdida de Validación por Épocas')
    axs[0].set_xlabel('Épocas')
    axs[0].set_ylabel('Pérdida')
    axs[0].axhline(y=mín_pérdida, color='darkslategray', linestyle='--')
    axs[0].legend()

    # trazar precisión por épocas
    axs[1].plot(épocas, valores_auc, 'bo', label='AUC de entrenamiento')
    axs[1].plot(épocas, val_valores_auc, 'cornflowerblue', label='AUC de validación')
    axs[1].plot(épocas, [AUC / 100] * len(épocas), 'darkmagenta', linestyle='--', label='AUC de prueba')
    axs[1].set_title('AUC de Validación por Épocas')
    axs[1].set_xlabel('Épocas')
    axs[1].set_ylabel('AUC')
    axs[1].axhline(y=máx_auc, color='darkslategray', linestyle='--')
    axs[1].legend()

    # calcular Matriz de Confusión
    cm = confusion_matrix(y_actual, y_pred)

    # crear gráfico de matriz de confusión
    im = axs[2].imshow(cm, interpolation='nearest', cmap=plt.cm.BuPu)
    axs[2].set_title(f"Matriz de Confusión \nAUC: {AUC:.2f}%")
    axs[2].set_xticks(np.arange(len(classes)))
    axs[2].set_yticks(np.arange(len(classes)))
    axs[2].set_xticklabels(classes, rotation=45)
    axs[2].set_yticklabels(classes)

    # recorrer la matriz, trazar cada elemento
    umbral = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        axs[2].text(j, i, format(cm[i, j], 'd'),
                     ha="center",
                     color="white" if cm[i, j] > umbral else "black")

    axs[2].set_ylabel('Etiqueta real')
    axs[2].set_xlabel('Etiqueta predicha')
    axs[2].grid(False)
    fig.tight_layout()
    plt.savefig(f"{model_name}.pdf")

    plt.show()


In [ ]:
# Esta función prepara los datos de prueba, obtiene predicciones del modelo y proporciona las etiquetas reales y predichas, 
# que pueden ser utilizadas para una evaluación adicional del rendimiento del modelo.

def prepare_for_test(model, test_gen):
    data, y_true = test_gen.next()
    y_pred_ = model.predict(data, batch_size = 64)
    y_pred = []
    for i in range(y_pred_.shape[0]):
        y_pred.append(np.argmax(y_pred_[i]))
        
    y_true = np.argmax(y_true, axis=1)
    
    return y_true, y_pred

In [ ]:
# Esta función sirve para visualizar un subconjunto de imágenes de un lote junto con sus etiquetas, 
# y opcionalmente, sus etiquetas predichas para comparación

def show_images(generator,y_pred=None):
    """
    Input: An image generator,predicted labels (optional)
    Output: Displays a grid of 9 images with lables
    """
    
    # get image lables
    labels =dict(zip([0,1,2,3], CLASSES))
    
    # get a batch of images
    x,y = generator.next()
    
    # display a grid of 9 images
    plt.figure(figsize=(10, 10))
    if y_pred is None:
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(x[i])
            plt.axis("off")
            plt.title("Class:{}".format(labels[np.argmax(y[i])]))
                                                     
    else:
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(x[i])
            plt.axis("off")
            plt.title("Actual:{} \nPredicted:{}".format(labels[np.argmax(y[i])],labels[y_pred[i]]))

#### Preprocesamiento de imágenes para el análisis

A continuación mostraremos la informacion de las imagenes y las clases tanto del directorio de train como de test.

In [ ]:
# Directory where the images are located
train_dir = "./data_sample/train"

# Load images and labels
X_train, y_train = read_data(train_dir)

# Show information about the loaded images
print("Number of loaded images:", len(X_train))
print("Dimensions of the image matrix:", X_train.shape)
print("Number of loaded labels:", len(y_train))
print("First 5 labels:", y_train[:5])  # Show the first 5 labels as an example

In [ ]:
# Directory where the images are located
test_dir = "./data_sample/test"

# Load images and labels
X_test, y_test = read_data(test_dir)

# Show information about the loaded images
print("Number of loaded images:", len(X_test))
print("Dimensions of the image matrix:", X_test.shape)
print("Number of loaded labels:", len(y_test))
print("First 5 labels:", y_test[:5])  # Show the first 5 labels as an example

In [ ]:
X_train.shape

In [ ]:
type(X_train[123])

##### Visualización de imágenes de muestra

A continuación, haciendo uso de la función generada para ello, pasamos a ver algunas de las imagenes de muestra.

In [ ]:
show_random_images_with_labels(X_train, y_train, num_images=10)

##### Visualización de la proporción de clases

Asimismo, procedemos a mirar el numero de imágenes por clase para ver si es un conjunto equilibrado.

In [ ]:
plt.figure(figsize=(5, 5))
plt.hist(y_train, bins=len(np.unique(y_train)), edgecolor='black', alpha=0.7)
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Number of Images per class in train dataset')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y',linestyle = "--", alpha=0.7)
plt.tight_layout()
plt.show()

### 2. Comparación de modelos

#### Modelo base: Random Forest

Construiremos un modelo de Random Forest como baseline.

In [ ]:
X_train = X_train/255
X_test = X_test/255

Primero aplicamos la funcion de aplanado:

In [ ]:
X_train_rf = flatten_gray_scale(X_train)
len(X_train_rf[0])

A continuócion aplicamos el shuffle (aleatoriedad):

In [ ]:
X_train_rf_shuffled, y_train_shuffled = shuffle(X_train_rf, y_train, random_state = 42)

Aplicamos el model:

In [ ]:
rf_clf = RandomForestClassifier()
np.mean(cross_val_score(rf_clf,X_train_rf_shuffled, y_train_shuffled, cv = 5, scoring = "accuracy"))

92% the accuracy debería ser lo mínimo.

#### Modelo convolucional simple (CNN model)

En este paso vamos a generar y probar un modelo de red convolucional simple diseñado "a mano".

En primer lugar vamos a definir ciertos directorios asi como semillas o seeds, valores que utilizaremos repetidamente a posteriori.

In [ ]:
base_dir = "./data_sample/"
root_dir = "./"
train_dir = base_dir + "train/"
test_dir = base_dir + "test/"

In [ ]:
CLASSES = [ 'NonDemented',
            'VeryMildDemented',
            'MildDemented',
            'ModerateDemented']

IMG_SIZE = 176
IMAGE_SIZE = [176, 176]
DIM = (IMG_SIZE, IMG_SIZE)

En el siguiente paso vamos a hacer uso de keras para la generación de los sets de entrenamiento, validación y test.

In [ ]:
datagen = IDG(rescale = 1./255, validation_split=0.1)

train_gen = datagen.flow_from_directory(directory=train_dir,
                                             target_size=DIM,
                                             batch_size=400,
                                             class_mode='categorical',
                                             subset='training',
                                             shuffle=True)

validation_gen = datagen.flow_from_directory(directory=train_dir,
                                             target_size=DIM,
                                             batch_size=400,
                                             class_mode='categorical',
                                             subset='validation',
                                             shuffle=True)

test_gen = datagen.flow_from_directory(directory=test_dir,
                                             target_size=DIM,
                                             batch_size=6400,
                                             class_mode='categorical')

Este de test con menos batch_size para el plot.

In [ ]:
test_gen_plot = datagen.flow_from_directory(directory=test_dir,
                                             target_size=DIM,
                                             batch_size=128,
                                             class_mode='categorical')

A continuación, generamos y entrenamos el modelo:

In [ ]:
# Create the model
model = Sequential()

# Add convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
# Add pooling layer
model.add(MaxPooling2D((2, 2)))

# Add another convolutional and pooling layers
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten for the dense layer
model.add(Flatten())

# Add dense layer
model.add(Dense(128, activation='relu'))

# Out layer with softmax activation for classification
model.add(Dense(len(CLASSES), activation='softmax'))

# Compile the model with personalized metrics
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[AUC(name='auc'), Precision(name='precision'), Recall(name='recall')])

# Train the model
history = model.fit(train_gen,
                    epochs=50,
                    validation_data=validation_gen)

Evaluamos el modelo:

In [ ]:
model.evaluate(test_gen)

Obtenemos valores necesarios para generar los plots:

In [ ]:
# Get the model predictions on the validation dataset.
y_pred_prob = model.predict(validation_gen)
y_pred = np.argmax(y_pred_prob, axis=1)

# Get the true labels from the validation dataset.
y_true = validation_gen.classes

Generamos el plot de las métricas:

In [ ]:
plot_training_metrics(history, model, test_gen_plot, y_true, y_pred, CLASSES, "model")

### Transfer learning

En este siguiente apartado, vamos a utilizar diferentes modelos de transfer learning, y finalmente haremos una comparativa de todos, para concluir cual podriamos mejorar para la tarea de predecir las clases de manera efectiva.

##### ResNet50

A continuación y de manera similar al apartado anterior, generamos, entrenamos el modelo, lo evaluamos y obtenemos las métricas.

In [ ]:
rn = ResNet50(input_shape=(IMG_SIZE,IMG_SIZE,3), weights='imagenet', include_top=False)
for layer in rn.layers:
    layer.trainable = False
x = Flatten()(rn.output)

prediction = Dense(4, activation='softmax')(x)

model = Model(inputs=rn.input, outputs=prediction)

model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc'),'acc'])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=8,
                                            restore_best_weights=True)

tic = time.perf_counter()
history = model.fit(train_gen,
                    steps_per_epoch=len(train_gen),
                    validation_data=validation_gen,
                    validation_steps=len(validation_gen),
                    epochs=50, callbacks=callback)
# time
toc = time.perf_counter()
print("Total Time:{}".format(round((toc-tic)/60,2)))

In [ ]:
y_true, y_pred = prepare_for_test(model, test_gen)

In [ ]:
plot_training_metrics(history, model, test_gen_plot, y_true, y_pred, CLASSES, model_name = "resnet50")

Finalmente guardamos el modelo:

In [ ]:
model_dir = "./alzheimer_resnet50_model"
model.save(model_dir, save_format='h5')
np.save('my_resnet50_history.npy', history.history)

##### InceptionV3

Hacemos lo exactamente lo mismo que para el modelo anterior:

In [ ]:
inception = InceptionV3(input_shape=(IMG_SIZE,IMG_SIZE,3), weights='imagenet', include_top=False)
for layer in inception.layers:
    layer.trainable = False
x = Flatten()(inception.output)

prediction = Dense(4, activation='softmax')(x)

model = Model(inputs=inception.input, outputs=prediction)

model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc'),'acc'])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=8,
                                            restore_best_weights=True)

tic = time.perf_counter()
history = model.fit(train_gen,
                    steps_per_epoch=len(train_gen),
                    validation_data=validation_gen,
                    validation_steps=len(validation_gen),
                    epochs=50, callbacks=callback)
# time
toc = time.perf_counter()
print("Total Time:{}".format(round((toc-tic)/60,2)))

In [ ]:
y_true, y_pred = prepare_for_test(model, test_gen)

In [ ]:
plot_training_metrics(history, model, test_gen_plot, y_true, y_pred, CLASSES, model_name = "inceptionv3")

In [ ]:
model_dir = "./alzheimer_inceptionv3_model"
model.save(model_dir, save_format='h5')
np.save('my_inceptionv3_history.npy', history.history)

##### DenseNet169

Hacemos exactamente lo mismo que para los modelos previos:

In [ ]:
dense = DenseNet169(input_shape=(IMG_SIZE,IMG_SIZE,3), weights='imagenet', include_top=False)
for layer in dense.layers:
    layer.trainable = False
x = Flatten()(dense.output)

prediction = Dense(4, activation='softmax')(x)

model = Model(inputs=dense.input, outputs=prediction)

model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc'),'acc'])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=8,
                                            restore_best_weights=True)

tic = time.perf_counter()
history = model.fit(train_gen,
                    steps_per_epoch=len(train_gen),
                    validation_data=validation_gen,
                    validation_steps=len(validation_gen),
                    epochs=50, callbacks=callback)
# time
toc = time.perf_counter()
print("Total Time:{}".format(round((toc-tic)/60,2)))

In [ ]:
y_true, y_pred = prepare_for_test(model, test_gen)

In [ ]:
plot_training_metrics(history, model, test_gen_plot, y_true, y_pred, CLASSES, model_name = "densenet169")

In [ ]:
model_dir = "./alzheimer_densenet169_model"
model.save(model_dir, save_format='h5')
np.save('my_densenet169_history.npy', history.history)

##### VGG16

Hacemos exactamente lo mismo que para los modelos previos:

In [ ]:
vgg = VGG16(input_shape=(IMG_SIZE,IMG_SIZE,3), weights='imagenet', include_top=False)
for layer in vgg.layers:
    layer.trainable = False
x = Flatten()(vgg.output)

prediction = Dense(4, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

model.compile(optimizer='adam',
loss=tensorflow.losses.CategoricalCrossentropy(),
metrics=[keras.metrics.AUC(name='auc'),'acc'])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=3,
                                            restore_best_weights=True)
tic = time.perf_counter()
history = model.fit(train_gen,
                    steps_per_epoch=len(train_gen),
                    validation_data=validation_gen,
                    validation_steps=len(validation_gen),
                    epochs=20, callbacks=callback)
# time
toc = time.perf_counter()
print("Total Time:{}".format(round((toc-tic)/60,2)))

In [ ]:
y_true, y_pred = prepare_for_test(model, test_gen)

In [ ]:
plot_training_metrics(history, model, test_gen_plot, y_true, y_pred, CLASSES, model_name = "vgg16")

In [ ]:
model_dir = "./alzheimer_vgg16_model"
model.save(model_dir, save_format='h5')
np.save('my_vgg16_history.npy', history.history)

#### Resumen y conclusiones, seleccionar el mejor modelo para ajustar

**En resumen:**

**CNN model:**

- AUC: 79.03%
- Accuracy: 58.87%
- Precision para NonDemented: 0.13, VeryMildDemented: 0.00, MildDemented: 0.47, ModerateDemented: 0.32
- Recall para NonDemented: 0.13, VeryMildDemented: 0.00, MildDemented: 0.46, ModerateDemented: 0.34
- F1-score para NonDemented: 0.13, VeryMildDemented: 0.00, MildDemented: 0.47, ModerateDemented: 0.33

**ResNet50:**

- AUC: 83.29%
- Accuracy: 53.95%
- Precision para NonDemented: 0.40, VeryMildDemented: 1.00, MildDemented: 0.83, ModerateDemented: 0.43
- Recall para NonDemented: 0.17, VeryMildDemented: 0.08, MildDemented: 0.45, ModerateDemented: 0.83
- F1-score para NonDemented: 0.24, VeryMildDemented: 0.15, MildDemented: 0.58, ModerateDemented: 0.57

**InceptionV3:**

- AUC: 84.00%
- Accuracy: 60.20%
- Precision para NonDemented: 0.73, VeryMildDemented: 1.00, MildDemented: 0.81, ModerateDemented: 0.48
- Recall para NonDemented: 0.20, VeryMildDemented: 0.17, MildDemented: 0.54, ModerateDemented: 0.86
- F1-score para NonDemented: 0.31, VeryMildDemented: 0.29, MildDemented: 0.65, ModerateDemented: 0.62

**DenseNet169:**

- AUC: 86.79%
- Accuracy: 62.39%
- Precision para NonDemented: 0.92, VeryMildDemented: 1.00, MildDemented: 0.61, ModerateDemented: 0.66
- Recall para NonDemented: 0.06, VeryMildDemented: 0.08, MildDemented: 0.95, ModerateDemented: 0.40
- F1-score para NonDemented: 0.12, VeryMildDemented: 0.15, MildDemented: 0.74, ModerateDemented: 0.50

**VGG16:**

- AUC: 89.00%
- Accuracy: 65.91%
- Precision para NonDemented: 0.83, VeryMildDemented: 1.00, MildDemented: 0.67, ModerateDemented: 0.63
- Recall para NonDemented: 0.11, VeryMildDemented: 0.17, MildDemented: 0.88, ModerateDemented: 0.57
- F1-score para NonDemented: 0.20, VeryMildDemented: 0.29, MildDemented: 0.76, ModerateDemented: 0.60

**Conclusiones hasta ahora:**

Como se mencionó anteriormente, para determinar qué modelo es el mejor, dado que las clases están altamente desbalanceadas, simplemente mirar la precisión puede no ser suficiente. Necesitamos considerar varios factores, accuracy, precision, recall, F1-score, and AUC (Area Under the Curve). Basándonos en estas métricas:

- **AUC and Accuracy**: VGG16 tiene el AUC más alto (89.00%) y precisión (65.91%) entre todos los modelos, lo que indica su capacidad para discriminar entre clases y su rendimiento predictivo general.

- **Precision and Recall**: VGG16 también muestra valores competitivos de precisión y sensibilidad en todas las clases en comparación con otros modelos. Sin embargo, todavía tiene dificultades con la sensibilidad para la clase 'NonDemented'.

- **F1-score**: VGG16 muestra puntuaciones F1 relativamente altas para la mayoría de las clases, lo que indica un buen equilibrio entre precisión y sensibilidad. Sin embargo, todavía tiene una puntuación F1 relativamente baja para la clase 'NonDemented'.

Considerando el rendimiento general en todas las métricas, **VGG16** parece ser el mejor modelo entre los proporcionados, mostrando el AUC y la precisión más altos, junto con valores competitivos de precisión, sensibilidad y puntuación F1 en la mayoría de las clases. Sin embargo, como identificar con precisión la clase 'No Demenciado' es crucial, se necesita una investigación adicional o ajustes al modelo.

Sin embargo, para este caso en el que seleccionar la clase precisa para diagnósticos y considerar el rendimiento general en todas las clases, **DenseNet169** parece ser la mejor opción ya que mantiene una precisión, sensibilidad y puntuación F1 relativamente altas en todas las clases. Logra un buen equilibrio entre identificar correctamente instancias de cada clase y minimizar las clasificaciones incorrectas. Por lo tanto, DenseNet169 podría ser el mejor modelo para seleccionar clases precisas para diagnósticos mientras se garantiza un rendimiento general.

**Agregar optimización del modelo o agregar data augmentation puede resultar en un mejor rendimiento en el conjunto de datos de prueba.**

--> **Vamos a mejorar los modelos VGG16 y DenseNet169**

### 3. Ajuste del modelo

#### Fine-tuning

**DenseNet169**:

Para mejorar el modelo, descongelamos las ultimas 10 capas:

In [ ]:
# Load pre-trained DenseNet169 model
dense = DenseNet169(input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet', include_top=False)

# Unfreeze the last few convolutional blocks for fine-tuning
for layer in dense.layers[:-10]:  # Unfreeze the last 10 layers for fine-tuning
    layer.trainable = True

# Flatten the output of DenseNet169 and add a dense layer for classification
x = Flatten()(dense.output)
prediction = Dense(4, activation='softmax')(x)

# Create the model
model = Model(inputs=dense.input, outputs=prediction)
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.AUC(name='auc'), 'acc'])

# Define early stopping callback
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                         patience=8,
                                         restore_best_weights=True)

# Start training
tic = time.perf_counter()
history = model.fit(train_gen,
                    steps_per_epoch=len(train_gen),
                    validation_data=validation_gen,
                    validation_steps=len(validation_gen),
                    epochs=50,
                    callbacks=callback
)
toc = time.perf_counter()

# Print total training time
print("Total Time: {}".format(round((toc-tic)/60, 2)))


In [ ]:
y_true, y_pred = prepare_for_test(model, test_gen)

In [ ]:
plot_training_metrics(history, model, test_gen_plot, y_true, y_pred, CLASSES, model_name = "dense_fine_tunning")

In [ ]:
model_dir = "./alzheimer_dense_fine_model"
model.save(model_dir, save_format='h5')
np.save('my_dense_fine_history.npy', history.history)

**VGG16**:

En este caso, congelamos todas las capas excepto el ultimo bloque convolucional:

In [ ]:

# Load pre-trained VGG16 model
vgg = VGG16(input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet', include_top=False)

# Freeze all layers except the last convolutional block
for layer in vgg.layers[:-4]:
    layer.trainable = False

# Flatten the output of VGG16 and add a dense layer for classification
x = Flatten()(vgg.output)
prediction = Dense(4, activation='softmax')(x)

# Create the model
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.AUC(name='auc'), 'acc'])

# Define early stopping callback
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                         patience=3,
                                         restore_best_weights=True)

# Start training
tic = time.perf_counter()
history = model.fit(train_gen,
                    steps_per_epoch=len(train_gen),
                    validation_data=validation_gen,
                    validation_steps=len(validation_gen),
                    epochs=30, callbacks=callback)
toc = time.perf_counter()

# Print total training time
print("Total Time: {}".format(round((toc-tic)/60, 2)))


In [ ]:
y_true, y_pred = prepare_for_test(model, test_gen)

In [ ]:
plot_training_metrics(history, model, test_gen_plot, y_true, y_pred, CLASSES, model_name = "vgg16_fine_tunning")

In [ ]:
model_dir = "./alzheimer_vgg16_fine_model"
model.save(model_dir, save_format='h5')
np.save('my_vgg16_fine_history.npy', history.history)

#### Aumento de datos (Data augmentation)

En este caso, realizamos data augmentation mediante ImageDataGenerator:

**DenseNet169**:

In [ ]:
# Load pre-trained DenseNet169 model
dense = DenseNet169(input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet', include_top=False)

# Freeze all layers except the last convolutional block
for layer in dense.layers:
    layer.trainable = False

# Flatten the output of DenseNet169 and add a dense layer for classification
x = Flatten()(dense.output)
prediction = Dense(4, activation='softmax')(x)

# Create the model
model = Model(inputs=dense.input, outputs=prediction)
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.AUC(name='auc'), 'acc'])

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Define early stopping callback
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                         patience=8,
                                         restore_best_weights=True)

# Start training
tic = time.perf_counter()
history = model.fit(
    train_datagen.flow(train_gen),
    steps_per_epoch=len(train_gen),
    validation_data=validation_gen,
    validation_steps=len(validation_gen),
    epochs=50,
    callbacks=callback
)
toc = time.perf_counter()

# Print total training time
print("Total Time: {}".format(round((toc-tic)/60, 2)))


In [ ]:
y_true, y_pred = prepare_for_test(model, test_gen)

In [ ]:
plot_training_metrics(history, model, test_gen_plot, y_true, y_pred, CLASSES, model_name = "dense_aug")

In [ ]:
model_dir = "./alzheimer_dense_aug"
model.save(model_dir, save_format='h5')
np.save('my_dense_aug_history.npy', history.history)

**VGG16**:

In [ ]:
# Load pre-trained VGG16 model
vgg = VGG16(input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet', include_top=False)

# Freeze all layers except the last convolutional block
for layer in vgg.layers[:-4]:
    layer.trainable = False

# Flatten the output of VGG16 and add a dense layer for classification
x = Flatten()(vgg.output)
prediction = Dense(4, activation='softmax')(x)

# Create the model
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.AUC(name='auc'), 'acc'])

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Define early stopping callback
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                         patience=3,
                                         restore_best_weights=True)

# Start training
tic = time.perf_counter()
history = model.fit(
    train_datagen.flow(train_gen),
    steps_per_epoch=len(train_gen),
    validation_data=validation_gen,
    validation_steps=len(validation_gen),
    epochs=30,
    callbacks=callback
)
toc = time.perf_counter()

# Print total training time
print("Total Time: {}".format(round((toc-tic)/60, 2)))

In [ ]:
y_true, y_pred = prepare_for_test(model, test_gen)

In [ ]:
plot_training_metrics(history, model, test_gen_plot, y_true, y_pred, CLASSES, model_name = "vgg16_aug")

In [ ]:
model_dir = "./alzheimer_vgg16_aug_model"
model.save(model_dir, save_format='h5')
np.save('my_vgg16_aug_history.npy', history.history)

Comparando los dos últimos modelos con los gnerados en primer lugar:

- DenseNet169 Fine-tuned supera a los modelos iniciales en cuanto a AUC, precisión, recall y F1-score, mostrando una mejora significativa en todas las métricas.
- VGG16 Fine-tuned también supera a los modelos iniciales, especialmente en términos de AUC, precisión, recall y F1-score. Sin embargo, la mejora no es tan sustancial como la de DenseNet169 Fine-tuned.

Por lo tanto, en comparación con los primeros modelos proporcionados, tanto DenseNet169 Fine-tuned como VGG16 Fine-tuned muestran mejoras significativas en las métricas de rendimiento, siendo **DenseNet169 Fine-tuned** el modelo que exhibe un rendimiento superior en general.

### 4. Evaluación del rendimiento en imágenes de prueba

A continuación, probamos a etiquetar las imágenes con el mejor modelo generado:

Cargamos el modelo:

In [ ]:
model_dir = "./alzheimer_dense_fine_model"
history_file = 'my_dense_fine_history.npy'
model = load_model(model_dir)
history = np.load(history_file, allow_pickle=True).item()

Obtenemos predicción:

In [ ]:
y_true, y_pred = prepare_for_test(model, test_gen)

Y por último visualizamos la predicciones:

In [ ]:
show_images(train_gen, y_pred)

### 5. Conclusiones

Basándonos en los resultados obtenidos al evaluar varios modelos de Deep Learning para la clasificación de etapas de la enfermedad de Alzheimer utilizando imágenes de resonancia magnética (MRI), se pueden obtener varias conclusiones para este proyecto:

**Rendimiento del modelo:**

- Se han evaluado diferentes arquitecturas de Deep Learning para la capacidad de discriminar entre las diferentes etapas de la enfermedad de Alzheimer utilizando imágenes de resonancia magnética.
- Entre los modelos evaluados, DenseNet169 y VGG16 han demostrado el mejor rendimiento en términos de AUC y precisión, especialmente después del fine-tuning.
- El fine-tuning de los modelos mejoró su rendimiento, lo que indica la importancia del aTransfer Learning y la mejora específica para las tareas de análisis de imágenes.

**Impacto en la detección temprana de la enfermedad:**

- El alto rendimiento de los modelos de Deep Learning desarrollados sugiere su utilidad potencial en la detección temprana de la enfermedad de Alzheimer.
- La detección temprana es crucial para intervenciones y manejo oportunos de la enfermedad, lo que potencialmente conduce a mejores resultados de tratamiento y una mejor calidad de vida para los pacientes.

**Implicaciones clínicas:**

- Implementar un modelo de aprendizaje profundo fiable para la clasificación de etapas de la enfermedad de Alzheimer podría ayudar a los médicos a realizar diagnósticos y decisiones de tratamiento más precisos.
- El modelo podría servir como una herramienta valiosa para radiólogos y neurólogos, ayudando en la interpretación de escáneres de MRI y proporcionando información adicional sobre la progresión de la enfermedad.

**Perspectivas futuras:**

- Futuras investigaciones podrían centrarse en mejorar los modelos y explorar modalidades de imágenes adicionales (por ejemplo, escáneres PET) para mejorar la precisión del diagnóstico y la clasificación de la enfermedad de Alzheimer.
- La colaboración con profesionales e instituciones de atención médica podría facilitar la integración de los modelos desarrollados en la práctica clínica, permitiendo un impacto y validación en el mundo real.

En resumen, el desarrollo y la evaluación exitosos de estos modelos para identificar patrones distintivos en imágenes de MRI de pacientes con Alzheimer representan un paso significativo hacia la mejora de la detección temprana de la enfermedad y los resultados de los pacientes en el campo de la neurología y la imagen médica.